
<h1> Using MOMENT for Imputation </h1>
<hr>

## Contents
### 1. A Quick Introduction to Imputation
### 2. Loading MOMENT
### 3. Inputs and Outputs
### 4. Zero-shot Imputation
#### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 4.1 Masking Time Series Patches
#### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 4.2 Imputation using MOMENT
#### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 4.3 Results
### 5. Example Code to Fine-tune MOMENT for Imputation

## 1. A Quick Introduction to Imputation

Multivariate time series are prevalent in real world applications such as finance, meteorology and healthcare. These time series often contain missing values due to various reasons, including equipment malfunctions and human errors. Missing values can impede the meaningful analyis of time series, and hence numerous studies have focused on the task of imputing these missing values using machine learning methodologies. In this tutorial, we will use MOMENT to address the issue of missing values. Formally, the imputation task can be defined as follows:

**Problem**: We are given a time series $T = [x_1, ..., x_L], \ x_i \in \mathbb{R}^{C}$ of length $L$ with $C$ channels (sensors or variables), and a boolean mask of same length $mask = [m_1, ..., m_L]$ indicating missing values. The objective of the imputation problem is to infer missing values and recover the original time series.

## 2. Loading MOMENT

We will first install the MOMENT package, load some essential packages and the pre-trained model. 

MOMENT can be loaded in 4 modes: (1) `reconstruction`, (2) `embedding`, (3) `forecasting`, and (4) `classification`.

In the `reconstruction` mode, MOMENT reconstructs input time series, potentially containing missing values. We can solve imputation and anomaly detection problems in this mode. This mode is suitable for solving imputation and anomaly detection tasks. During pre-training, MOMENT is trained to predict the missing values within uniformly randomly masked patches (disjoint sub-sequences) of the input time series, leveraging information from observed data in other patches. As a result, MOMENT comes equipped with a pre-trained reconstruction head, enabling it to address imputation and anomaly detection challenges in a zero-shot manner! Check out the `anomaly_detection.ipynb` notebook for more details!

In the `embedding` model, MOMENT learns a $d$-dimensional embedding (e.g., $d=1024$ for `MOMENT-1-large`) for each input time series. These embeddings can be used for clustering and classification. MOMENT can learn embeddings in a zero-shot setting! Check out `representation_learning.ipynb` and `classification.ipynb` notebooks for more details! 

The `forecasting` and `classification` modes are used for forecasting and classification tasks, respectively. In these modes, MOMENT learns representations which are subsequently mapped to the forecast horizon or the number of classes, using linear forecasting and classification heads. Both the forecasting and classification head are randomly initialized, and therefore must be fine-tuned before use. Check out the `forecasting.ipynb` and `classification.ipynb` notebooks for more details!

In [1]:
!pip install numpy pandas scikit-learn matplotlib tqdm
!pip install git+https://github.com/moment-timeseries-foundation-model/moment.git

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/moment-timeseries-foundation-model/moment.git to /tmp/pip-req-build-o7mr28d4
  Running command git clone --filter=blob:none --quiet https://github.com/moment-timeseries-foundation-model/moment.git /tmp/pip-req-build-o7mr28d4
  Resolved https://github.com/moment-timeseries-foundation-model/moment.git to commit fb620934ef6b67f878bc21b8640b22d117dbffa6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for momentfm: filename=momentfm-0.1.2-py3-none-any.whl size=33793 sha256=0dbe7763fb439739d5c2fceccd6dade8d18e25c53ac61c76ece231408f6070da
  Stored in directory: /tmp/pip-ephem-wheel-cache-ykkcx6gr/wheels/6e/62/da/a5752a90276fbaebe0da5c0d02272f6549f43e9ceaac72907e
Successfully built momentfm


In [2]:
from momentfm.utils.utils import control_randomness
control_randomness(seed=13) # Set random seeds for PyTorch, Numpy etc.

/usr/local/share/miniconda3/envs/env/lib/python3.10/site-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [3]:
from momentfm import MOMENTPipeline

model = MOMENTPipeline.from_pretrained(
    "AutonLab/MOMENT-1-large",
    model_kwargs={'task_name': 'reconstruction'} # For imputation, we will load MOMENT in `reconstruction` mode
    # local_files_only=True,  # Whether or not to only look at local files (i.e., do not try to download the model).
)

config.json:   0%|          | 0.00/951 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.39G [00:00<?, ?B/s]

In [4]:
model.init()
print(model)

MOMENTPipeline(
  (normalizer): RevIN()
  (tokenizer): Patching()
  (patch_embedding): PatchEmbedding(
    (value_embedding): Linear(in_features=8, out_features=1024, bias=False)
    (position_embedding): PositionalEmbedding()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
  

In [5]:
# Number of parameters in the encoder
num_params = sum(p.numel() for p in model.encoder.parameters())
print(f"Number of parameters: {num_params}")

Number of parameters: 341231104


## 3. Inputs and Outputs

Let's begin by performing a forward pass through MOMENT and examining its outputs!

MOMENT takes 3 inputs: 
1. An input time series of length $T=512$ timesteps and $C$ channels, and 
2. Two optional masks, both of length $T=512$. 
    - The input mask is utilized to regulate the time steps or patches that the model should attend to. For instance, in the case of shorter time series, you may opt not to attend to padding. To implement this, you can provide an input mask with zeros in the padded locations.  
    - The second mask, referred to simply as mask, denotes masked or unobserved values. We employ mask tokens to replace all patches containing any masked time step (for further details, refer to Section 3.2 in our [paper](https://arxiv.org/abs/2402.03885)). MOMENT can attend to these mask tokens during reconstruction.
    - By default, all time steps are observed and attended to.

MOMENT returns a `TimeseriesOutputs` object. Since this is a reconstruction task, it returns a `reconstruction` of the input. 

In [6]:
from pprint import pprint
import torch

# takes in tensor of shape [batch_size, n_channels, context_length]
x = torch.randn(16, 1, 512)
output = model(x_enc=x)
pprint(output)

/usr/local/share/miniconda3/envs/env/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/share/miniconda3/envs/env/lib/python3.10/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


TimeseriesOutputs(forecast=None,
                  anomaly_scores=None,
                  logits=None,
                  labels=None,
                  input_mask=tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]]),
                  pretrain_mask=tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]),
                  reconstruction=tensor([[[-0.0437, -0.0208,  0.2231,  ...,  0.2506, -0.0512,  0.0481]],

        [[-0.0950,  0.0425,  0.0427,  ..., -0.0169, -0.0334, -0.1129]],

        [[ 0.0962, -0.0044,  0.0703,  ..., -0.0681,  0.0438, -0.1940]],

        ...,

        [[-0.2346, -0.0597,  0.0352,  ...,  0.2396,  0.1217,  0.0

## 4. Zero-shot Imputation

Now we'll show you how to use MOMENT to do zero-shot imputation!

In these experiments, we will use Hourly Electricity Transformer Temperature (ETTh1) dataset introduced by [Zhou et al., 2020](https://arxiv.org/abs/2012.07436). Check out [ETDataset](https://github.com/zhouhaoyi/ETDataset) for more information! We will use the ETTh1 dataset since missing values are common in this domain!

We'll start by reading and pre-processing this dataset using the `InformerDataset` class. Since we can do zero-shot imputation, we will just load the testing part of this data. Note that MOMENT was not exposed to the testing part of this dataset during pre-training. 

In [13]:
import pandas as pd
import functools
from momentfm.data.informer_dataset import InformerDataset
from torch.utils.data import DataLoader

In [17]:
# Función de parche sin 'copy'
def infer_objects_fixed(df):
    return pd.DataFrame.infer_objects(df)  # Sin el argumento copy

# Hook temporal para modificar el DataFrame antes de su uso
def patch_dataframe(df):
    if hasattr(df, "infer_objects"):  
        df.infer_objects = lambda: infer_objects_fixed(df)  # Reemplaza solo en esta instancia

# Hook en `InformerDataset`
class PatchedInformerDataset(InformerDataset):
    def _read_data(self):
        super()._read_data()
        patch_dataframe(self.data)  # Parche en la instancia del DataFrame

# Usa la versión parcheada
test_dataset = PatchedInformerDataset(
    data_split='test', 
    task_name='imputation', 
    data_stride_len=512
)

test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)


RecursionError: maximum recursion depth exceeded

Now let's visualize the time series

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

n_channels = test_dataset[0][0].shape[0]
idx = np.random.randint(0, len(test_dataset))
channel_idx = np.random.randint(0, n_channels)
plt.plot(test_dataset[idx][0][channel_idx, :].squeeze(), c='darkblue')
plt.title(f'idx={idx}  | channel={channel_idx}')
plt.show()

### 4.1 Masking Time Series Patches

Since there are no missing values in this dataset, we will randomly mask time series subsequences to evaluate MOMENT's ability to reason about missing values. Instead of masking individual time steps at random, we will mask time series patches uniformly at random using the `Masking` class.

In [ ]:
from momentfm.utils.masking import Masking

mask_generator = Masking(mask_ratio=0.25) # Mask 25% of patches randomly 

### 4.2 Imputation using MOMENT

Since there are no missing values in this dataset, we will randomly mask time series subsequences to evaluate MOMENT's ability to reason about missing values. Instead of masking individual time steps at random, we will mask time series patches uniformly at random using the `Masking` class.

In [ ]:
from tqdm import tqdm

device = "cuda:1" if torch.cuda.is_available() else "cpu"
model = model.to(device).float()

trues, preds, masks = [], [], []
with torch.no_grad():
    for batch_x, batch_masks in tqdm(test_dataloader, total=len(test_dataloader)):
        trues.append(batch_x.numpy())
        
        batch_x = batch_x.to(device).float()
        n_channels = batch_x.shape[1]
        
        # Reshape to [batch_size * n_channels, 1, window_size]
        batch_x = batch_x.reshape((-1, 1, 512)) 
        
        batch_masks = batch_masks.to(device).long()
        batch_masks = batch_masks.repeat_interleave(n_channels, axis=0)
        
        mask = mask_generator.generate_mask(
            x=batch_x, input_mask=batch_masks).to(device).long()

        output = model(x_enc=batch_x, input_mask=batch_masks, mask=mask) # [batch_size, n_channels, window_size]
        
        reconstruction = output.reconstruction.detach().cpu().numpy()
        mask = mask.detach().squeeze().cpu().numpy()
        
        # Reshape back to [batch_size, n_channels, window_size]
        reconstruction = reconstruction.reshape((-1, n_channels, 512)) 
        mask = mask.reshape((-1, n_channels, 512))
                
        preds.append(reconstruction)
        masks.append(mask)

preds = np.concatenate(preds)
trues = np.concatenate(trues)
masks = np.concatenate(masks)

print(f"Shapes: preds={preds.shape} | trues={trues.shape} | masks={masks.shape}")

### 4.3 Results

Since there are no missing values in this dataset, we will randomly mask time series subsequences to evaluate MOMENT's ability to reason about missing values. Instead of masking individual time steps at random, we will mask time series patches uniformly at random using the `Masking` class.

In [ ]:
from momentfm.utils.forecasting_metrics import mse, mae

print(f"Mean Squarred Error (MSE)={mse(y=trues[masks==0], y_hat=preds[masks==0], reduction='mean')}")
print(f"Mean Absolute Error (MAE)={mae(y=trues[masks==0], y_hat=preds[masks==0], reduction='mean')}")

Now let's visualize random time series windows! White patches are masked and black patches are observed.

In [ ]:
idx = np.random.randint(trues.shape[0])
channel_idx = np.random.randint(trues.shape[1])

fig, axs = plt.subplots(2, 1, figsize=(10, 5))
axs[0].set_title(f"Channel={channel_idx}")
axs[0].plot(trues[idx, channel_idx, :].squeeze(), label='Ground Truth', c='darkblue')
axs[0].plot(preds[idx, channel_idx, :].squeeze(), label='Predictions', c='red')
axs[0].legend(fontsize=16)

axs[1].imshow(np.tile(masks[np.newaxis, idx, channel_idx], reps=(8, 1)), cmap='binary')
plt.show()

## 5. Example Code to Fine-tune MOMENT for Imputation

To improve MOMENT's imputation performance, you can fine-tune it on any dataset. In our [paper](https://arxiv.org/abs/2402.03885), we fine-tune the final reconstruction head, but you can also fine-tune the entire model on your data. Here is some example code:

```python

# Optimize Mean Squarred Error using your favourite optimizer
criterion = torch.nn.MSELoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

mask_generator = Masking(mask_ratio=0.3) # Mask 30% of patches randomly 

for batch_x, batch_masks in tqdm(test_dataloader, total=len(test_dataloader)):
    n_channels = batch_x.shape[1]
    
    # Reshape to [batch_size * n_channels, 1, window_size]
    batch_x = batch_x.reshape((-1, 1, 512)) 
    
    batch_masks = batch_masks.to(device).long()
    batch_masks = batch_masks.repeat_interleave(n_channels, axis=0)
    
    # Randomly mask some patches of data
    mask = mask_generator.generate_mask(
        x=batch_x, input_mask=batch_masks).to(device).long()

    # Forward
    output = model(x_enc=batch_x, input_mask=batch_masks, mask=mask) 
    
    # Compute loss
    recon_loss = criterion(output.reconstruction, original)
    observed_mask = batch_masks * (1 - mask)
    masked_loss = observed_mask * recon_loss
    
    loss = masked_loss.nansum() / (observed_mask.nansum() + 1e-7)
    
    print(f"loss: {loss.item()}")
    
    # Backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
```